In [ ]:
import os
import sys
import random

import numpy as np
import cv2

In [ ]:
#Data generator. Can read a dataset of images and masks and can export compressed images.

class DataGen():
    def __init__(self, ids, path, path2, batch_size=8, image_size=128):
        self.ids = ids
        self.path = path
        self.path2 = path2
        self.batch_size = batch_size
        self.image_size = image_size
        
    def __load__(self, id_name):
        ## Path
        
        image_path = os.path.join(self.path, id_name)

        mask_path = os.path.join(self.path2, id_name[:-4] + "_segmentation.png")
        
        ## Reading Images
        image = cv2.imread(image_path)
        image = cv2.resize(image, (self.image_size, self.image_size))
        
        ## Reading Masks
        mask = cv2.imread(mask_path, -1)
        mask = cv2.resize(mask, (self.image_size, self.image_size))
        mask = np.expand_dims(mask, axis=-1)
        mask = np.maximum(mask, mask)
        
        ## Normalizaing 
        image = image/255.0
        mask = mask/255.0
        
        return image, mask
    
    def __saveitem__(self, id_name, location):
        _img, _mask = self.__load__(id_name)
        cv2.imwrite('dataset/'+location+'/Segmentazione/ISIC2018_Task1-2_Training_Input/' + id_name, _img*255)
        cv2.imwrite('dataset/'+location+'/Segmentazione/ISIC2018_Task1_Training_GroundTruth/' + id_name[:-4] + "_segmentation.png", _mask*255)


In [ ]:
image_size = 256

# Select dataset
sel_dat = "Dataset_Nei"

train_path = "dataset/" + sel_dat + "/Segmentazione/ISIC2018_Task1-2_Training_Input"
gt_path = "dataset/" + sel_dat + "/Segmentazione/ISIC2018_Task1_Training_GroundTruth"

# Training Ids
img_ids = next(os.walk(train_path))
img_ids = img_ids[2:][0]
gt_ids = next(os.walk(gt_path))
gt_ids = gt_ids[2:][0]

gen = DataGen(img_ids, train_path, gt_path, batch_size=1, image_size=image_size)

In [ ]:
#create the directory tree if it doesn't exist

directory = "dataset/Dataset_compressed_" + str(image_size)

try:
    os.stat(directory)
except:
    os.mkdir(directory)
    os.mkdir(directory + "/Segmentazione")
    os.mkdir(directory + "/Segmentazione/ISIC2018_Task1-2_Training_Input")
    os.mkdir(directory + "/Segmentazione/ISIC2018_Task1_Training_GroundTruth")


In [ ]:
#saves a the dataset in the /Dataset_compressed_[size] folder. Size depending on image_size

for i in range(0,len(img_ids)):
    if img_ids[i][:-4]+"_segmentation.png" in gt_ids:
        gen.__saveitem__(img_ids[i], "Dataset_compressed_" + str(image_size))
    else:
        text = "could not save " + img_ids[i]
        print(img_ids[i])